In [ ]:
from bs4 import BeautifulSoup
from google.colab import files
import requests
import numpy as np
import pandas as pd

The goal of this project is to webscrape Meta's newsroom for articles tagged with

* Safety and Expression
* Combatting Misinformation
* Data and Privacy

# Safety and Expression

In [ ]:
# Get article meta data: date, updated date, link, title of article

data = []

for i in range(0,21):
  request = requests.get(url=f"https://about.fb.com/news/category/safety-and-expression/page/{i}/")
  parser = BeautifulSoup(request.content, "html.parser")

  area_section = parser.find("div", class_="archive-articles-container")
  article_section = area_section.find_all("div", class_="uk-width-3-5 article-excerpt")
  for article in article_section:
    try:
        header = article.find("header", class_="entry-header")
        link_source = header.find("a", href=True)

        article_title = link_source.get_text()
        link = link_source.get("href")

        date_source = header.find("time", class_="entry-date published")
        date_source2 = header.find("time", class_="updated")
        date = date_source.get_text()
        date2 = date_source2.get_text()

        data.append((link, article_title, date, date2))

    except:
        header = article.find("header", class_="entry-header")
        link_source = header.find("a", href=True)

        article_title = link_source.get_text()
        link = link_source.get("href")

        date_source = header.find("time", class_="entry-date published updated")
        date = date_source.get_text()
        date2 = None

        data.append((link, article_title, date, date2))


article_meta_ds = pd.DataFrame(data, columns=["Link", "Article Title" , "Date", "Updated Date"])

print(article_meta_ds)

                                                  Link  \
0    https://about.fb.com/news/2024/09/instagram-te...   
1    https://about.fb.com/news/2024/09/preventing-s...   
2    https://about.fb.com/news/2024/07/combating-fi...   
3    https://about.fb.com/news/2024/04/new-tools-to...   
4    https://about.fb.com/news/2024/02/helping-teen...   
..                                                 ...   
205  https://about.fb.com/news/2019/01/a-discussion...   
206  https://about.fb.com/news/2019/01/oversight-bo...   
207  https://about.fb.com/news/2019/01/what-kind-of...   
208  https://about.fb.com/news/2019/01/banning-twin...   
209  https://about.fb.com/news/2018/12/content-revi...   

                                         Article Title                Date  \
0    Introducing Instagram Teen Accounts: Built-In ...  September 17, 2024   
1    Preventing Suicide and Self-Harm Content Sprea...  September 12, 2024   
2    Combating Financial Sextortion Scams From Nigeria       July 24,

In [ ]:
# Get article body text and zip this with article_meta_ds
# This process takes 2 minutes

scraped_dataset_safety = [ ]

for idx, row in article_meta_ds.iterrows():
  article_request = requests.get(url=row['Link'])
  parser = BeautifulSoup(article_request.content, "html.parser")

  text_section = parser.find("div", class_="uk-width-2-3@m article-container")
  paragraphs = text_section.find_all("p")

  cleaned_text = [para.get_text(strip=True) for para in paragraphs]
  scraped_dataset_safety.append(cleaned_text)
  print(idx, cleaned_text)



0 ['Today, we’re introducing Instagram Teen Accounts, a new experience for teens, guided by parents. Teen Accounts have built-in protections which limit who can contact them and the content they see, and also provide new ways for teens to explore their interests. We’ll automatically place teens into Teen Accounts, and teens under 16 will need a parent’s permission to change any of these settings to be less strict.', 'We know parents want to feel confident that their teens can use social media to connect with their friends and explore their interests, without having to worry about unsafe or inappropriate experiences. We understand parents’ concerns, and that’s why we’re reimagining our apps for teens with new Teen Accounts. This new experience is designed to better support parents, and give them peace of mind that their teens are safe with the right protections in place. Teens will also get access to a new feature, made just for them, that lets them select topics they want to see more o

In [ ]:
article_meta_ds["Text"] = scraped_dataset_safety
article_meta_ds.to_excel('meta_content_safety.xlsx', index=False)
files.download('meta_content_safety.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Combatting Misinformation

In [ ]:
data2 = []
for i in range(0, 8):
  request = requests.get(url=f"https://about.fb.com/news/tag/misinformation/page/{i}/")
  parser = BeautifulSoup(request.content, "html.parser")

  area_section = parser.find("div", class_="archive-articles-container")
  article_section = area_section.find_all("div", class_="uk-width-3-5 article-excerpt")
  for article in article_section:
    try:
        header = article.find("header", class_="entry-header")
        link_source = header.find("a", href=True)

        article_title = link_source.get_text()
        link = link_source.get("href")

        date_source = header.find("time", class_="entry-date published")
        date_source2 = header.find("time", class_="updated")
        date = date_source.get_text()
        date2 = date_source2.get_text()

        data2.append((link, article_title, date, date2))
    except:
        header = article.find("header", class_="entry-header")
        link_source = header.find("a", href=True)

        article_title = link_source.get_text()
        link = link_source.get("href")

        date_source = header.find("time", class_="entry-date published updated")
        date = date_source.get_text()
        date2 = None

        data2.append((link, article_title, date, date2))

article_meta_ds_2 = pd.DataFrame(data2, columns=["Link", "Article Title" , "Date", "Updated Date"])

print(article_meta_ds_2)

                                                 Link  \
0   https://about.fb.com/news/2022/11/metas-progre...   
1   https://about.fb.com/news/2022/08/how-meta-is-...   
2   https://about.fb.com/news/2022/07/oversight-bo...   
3   https://about.fb.com/news/2022/07/how-metas-pr...   
4   https://about.fb.com/news/2022/02/metas-ongoin...   
..                                                ...   
74  https://about.fb.com/news/2018/10/removing-ina...   
75  https://about.fb.com/news/2018/09/expanding-fa...   
76  https://about.fb.com/news/2018/06/hard-questio...   
77  https://about.fb.com/news/2018/05/facing-facts...   
78  https://about.fb.com/news/2018/05/inside-feed-...   

                                        Article Title                Date  \
0    Sharing Our Progress on Combating Climate Change    November 6, 2022   
1   How Meta Is Preparing for Brazil’s 2022 Elections     August 12, 2022   
2   Meta Asks Oversight Board to Advise on COVID-1...       July 26, 2022   
3   How

In [ ]:
scraped_dataset_misinfo = []

for idx, row in article_meta_ds_2.iterrows():
  article_request = requests.get(url=row['Link'])
  parser = BeautifulSoup(article_request.content, "html.parser")

  text_section = parser.find("div", class_="uk-width-2-3@m article-container")
  paragraphs = text_section.find_all("p")

  cleaned_text = [para.get_text(strip=True) for para in paragraphs]
  scraped_dataset_misinfo.append(cleaned_text)
  print(idx, cleaned_text)

0 ['Update on November 11, 2022 at 8:00AM PT:', 'Meta has become afounding member of the Asia Clean Energy Coalition,which will advocate for eliminating barriers to corporate renewable energy purchasing in Asia.', 'Originally published on November 6, 2022 at 11:00PM PT:', 'As leaders and experts convene in Egypt for COP27, we’re outlining progress on our climate commitments and announcing expanded tools to help our communities take action against climate change. Like previous years, we’re focusing on how we can leverage our technologies, apps and operational strategy to demonstrate our commitment to solutions.', 'Since 2020, we’ve achieved net zero greenhouse gas emissions for our global operations as they are now supported by 100% renewable energy. We’ve also set ambitious goals to achieve net zero value chain emissions and to restore more water than we consume in our global operations in 2030. Reducing our emissions and helping our suppliers to do the same is our top priority to reac

In [ ]:
article_meta_ds_2['Text'] = scraped_dataset_misinfo

article_meta_ds_2.to_excel('meta_content_misinfo.xlsx', index=False)

files.download('meta_content_misinfo.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Data and Privacy

In [57]:
data3 = []

for i in range(0, 17):
  request = requests.get(url=f"https://about.fb.com/news/category/data-and-privacy/page/{i}/")
  parser = BeautifulSoup(request.content, "html.parser")

  area_section = parser.find("div", class_="archive-articles-container")
  article_section = area_section.find_all("div", class_="uk-width-3-5 article-excerpt")
  for article in article_section:
    try:
        header = article.find("header", class_="entry-header")
        link_source = header.find("a", href=True)

        article_title = link_source.get_text()
        link = link_source.get("href")

        date_source = header.find("time", class_="entry-date published")
        date_source2 = header.find("time", class_="updated")
        date = date_source.get_text()
        date2 = date_source2.get_text()

        data3.append((link, article_title, date, date2))
    except:
        header = article.find("header", class_="entry-header")
        link_source = header.find("a", href=True)

        article_title = link_source.get_text()
        link = link_source.get("href")

        date_source = header.find("time", class_="entry-date published updated")
        date = date_source.get_text()
        date2 = None

        data3.append((link, article_title, date, date2))

article_meta_ds_3 = pd.DataFrame(data3, columns=["Link", "Article Title" , "Date", "Updated Date"])

print(article_meta_ds_3)

                                                  Link  \
0    https://about.fb.com/news/2024/03/end-to-end-e...   
1    https://about.fb.com/news/2024/01/investing-in...   
2    https://about.fb.com/news/2023/12/default-end-...   
3    https://about.fb.com/news/2023/11/new-tools-to...   
4    https://about.fb.com/news/2023/10/collaboratin...   
..                                                 ...   
163  https://about.fb.com/news/2019/02/addressing-c...   
164  https://about.fb.com/news/2019/01/a-discussion...   
165  https://about.fb.com/news/2019/01/data-privacy...   
166  https://about.fb.com/news/2019/01/designing-se...   
167  https://about.fb.com/news/2019/01/what-kind-of...   

                                         Article Title               Date  \
0         End-to-End Encryption on Messenger Explained     March 28, 2024   
1                                 Investing In Privacy   January 25, 2024   
2    Launching Default End-to-End Encryption on Mes...   December 6, 202

In [ ]:
scraped_dataset_privacy = []

for idx, row in article_meta_ds_3.iterrows():
  article_request = requests.get(url=row['Link'])
  parser = BeautifulSoup(article_request.content, "html.parser")

  text_section = parser.find("div", class_="uk-width-2-3@m article-container")
  paragraphs = text_section.find_all("p")

  cleaned_text = [para.get_text(strip=True) for para in paragraphs]
  scraped_dataset_privacy.append(cleaned_text)
  print(idx, cleaned_text)

In [ ]:
article_meta_ds_3['Text'] = scraped_dataset_privacy
article_meta_ds_3.to_excel('meta_content_privacy.xlsx', index=False)
files.download('meta_content_privacy.xlsx')